In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import joblib
from PIL import Image
from datetime import datetime, timedelta


import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import math
import matplotlib.pyplot as plt
import warnings
from sklearn.decomposition import PCA
import joblib

In [2]:


def compare_columns_df(deploy_df, target_df):
    # Find missing columns from target_df compared to deploy_df
    missing_from_target = set(target_df.columns) - set(deploy_df.columns)
    
    # Find missing columns from deploy_df compared to target_df
    missing_from_deploy = set(deploy_df.columns) - set(target_df.columns)

    # Output the missing columns and their counts
    print("Missing columns from deploy_df compared to target_df:", missing_from_target)
    print("Number of missing columns from deploy_df compared to target_df:", len(missing_from_target))
    
    print("Missing columns from target_df compared to deploy_df:", missing_from_deploy)
    print("Number of missing columns from target_df compared to deploy_df:", len(missing_from_deploy))

    # Check if the lengths of the DataFrames' columns are equal
    print("Number of columns in deploy_df:", len(deploy_df.columns))
    print("Number of columns in target_df:", len(target_df.columns))
    
    if len(deploy_df.columns) == len(target_df.columns):
        print("The number of columns in deploy_df is equal to that in target_df.")
    else:
        print("The number of columns in deploy_df is not equal to that in target_df.")



In [3]:
# Load datasets
def load_dataset(path):
    df = pd.read_csv(path)
    if "time" in df.columns:
        df["time"] = pd.to_datetime(df["time"], utc = True)
    return df

def manipulate_datasets(energy_df, weather_df):
    #deal with null values
    #Energy Features
    energy_df["time"] = pd.to_datetime(energy_df["time"], utc = True)
    weather_df["time"] = pd.to_datetime(weather_df["time"], utc = True)
    energy_generation_columns = ['generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil gas', 'generation fossil hard coal',
       'generation fossil oil', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation nuclear',
       'generation other', 'generation other renewable', 'generation solar',
       'generation waste', 'generation wind onshore'
]
    energy_df["total_gen"] = energy_df[energy_generation_columns].sum(axis=1)

    #Weather data features creation
    weather_df["temp_diff"] = weather_df["temp_max"] - weather_df["temp_min"]
    weather_df.drop(columns=["temp_max","temp_min"],inplace=True)
    weather_df["weather_id"] = weather_df["weather_id"].astype(str)
    weather_df = pd.get_dummies(weather_df, columns=["weather_id"], prefix="weather_id")
    #Combine the dfs
    cities = weather_df["city_name"].unique()
    user_data = energy_df.copy()
    for city in cities:
        #indexes values for a certain city name and drops the column
        df_city = weather_df[weather_df["city_name"] == city]
        df_city = df_city.drop(columns="city_name")
        
        #Duplicate times still exist within the weather dataset with a unique city. To gain a unique time, a 
        numeric_cols = df_city.select_dtypes(include=['number']).columns
        
        #Assigns the mean of duplicate time value entries to a unique time value
        df_city = df_city.groupby('time')[numeric_cols].agg('mean').reset_index()
        
        #Rename columns to include the city name
        for col in numeric_cols:
            df_city = df_city.rename(columns={col: col + "_" + city})
        
        #Merge the dfs
        user_data = user_data.merge(df_city, on="time", how="inner")
    #Drop columns as no correlation exists 
    try:
        user_data.drop(['snow_3h_ Barcelona', 'snow_3h_Seville'], axis=1, inplace=True)
    except:
        raise KeyError(f"Ensure all five cities are represented within the dataset")
    #Fill nan values
    user_data = user_data.fillna(user_data.mean())

    return user_data

def create_engineered_features(df, feature_engineered_data): 
    #Time Feats
    warnings.filterwarnings("ignore")
    def create_time_features(df, create_time_period_feature = False, step = 6):
        #Creates time related features of month, day, hour
        time_feat_df = pd.DataFrame()
        time_feat_df = df[["time","price actual"]]

        time_feat_df["month"] = time_feat_df['time'].dt.month_name()
        time_feat_df["day"] = time_feat_df['time'].dt.day_name()
        time_feat_df["hour"] = time_feat_df["time"].dt.hour.astype('category')
        #Creates stepped features based on hour
        #using pd.getdummies creates 23 additional features, whereas hour features could be captured via a period
        if create_time_period_feature:
            for i in range(step, 24, step):
                time_feat_df[f"is_btwn_{i}_and_{i+step}"] = time_feat_df["hour"].isin(range(i, i + step)).astype(bool)

            time_feat_df.drop(columns=["hour"],inplace=True)
        
        return time_feat_df
    time_best_feat = create_time_features(df, create_time_period_feature=True,step = 6).drop(columns="price actual")

#Lag and MA feats
# Create time features
    lag_ma_feats = create_time_features(df)
    lag_ma_feats["year"] = lag_ma_feats["time"].dt.year
    lag_ma_feats["week"] = lag_ma_feats["time"].dt.isocalendar().week
    lag_ma_feats.set_index("time", inplace=True)

    # Compute weekly average and merge with previous years for comparison
   
    weekly_avg_last_year = lag_ma_feats.groupby(["year", "week"])["price actual"].mean().reset_index()
    weekly_avg_last_year["last_year_weekly_mean_price"] = weekly_avg_last_year.groupby("week")["price actual"].shift(1)
    # Determine if a "last year exists within data, if it does not fill with mean from df_train"   
    if df.loc[df.index[-1], "time"] - timedelta(days=365) not in df["time"].values:
        weekly_avg_last_year["last_year_weekly_mean_price"].fillna(feature_engineered_data.groupby(feature_engineered_data["time"].dt.isocalendar().week)["last_year_weekly_mean_price"].transform("mean"), inplace=True)
    else:
        weekly_avg_last_year["last_year_weekly_mean_price"].fillna(weekly_avg_last_year.groupby("week")["last_year_weekly_mean_price"].transform("mean"), inplace=True)

    # Create lagged features
    for lag in [24, 24*7]: 
        lag_ma_feats[f"prev_{'day' if lag == 24 else 'week'}_price"] = lag_ma_feats["price actual"].shift(lag)
        lag_ma_feats[f"prev_{'day' if lag == 24 else 'week'}_price"].fillna(lag_ma_feats["prev_day_price"].mean(), inplace=True)

    # Create moving averages
    for period in [1, 3, 7, 14, 30, 90]:
        lag_ma_feats[f"{period}_day_ma"] = lag_ma_feats["price actual"].rolling(f"{period}D", min_periods=1).mean()

    lag_ma_feats = lag_ma_feats.reset_index().merge(weekly_avg_last_year.drop(columns="price actual"), on=["year", "week"], how="outer")
    lag_ma_feats.drop(columns=['month', 'day','hour','year', 'week'], inplace=True)
    #drop response variable
    lag_ma_feats.drop(columns="price actual",inplace=True)
    merged_time_lag_ma_feat = pd.merge(time_best_feat,lag_ma_feats,on="time",how="inner")
    #merge with org. df
    df = df.merge(merged_time_lag_ma_feat,on="time",how="inner")
    
    #Create quadratic trend features
    df.reset_index(inplace=True)
    df.rename(columns={"index":"t"},inplace=True)
    df["t^2"] = df["t"]**2

    df = pd.get_dummies(df,drop_first=True)
    return df

def apply_transformations(user_data, train_df):
    #Int transformers
    transformer_scaler = joblib.load(r"transformers\scaler.pkl")
    transformer_pca = joblib.load(r"transformers\pca.pkl")

    #Response variable
    time = user_data["time"]
    y = user_data["price actual"]
    user_data.drop(columns = ["time","price actual"], inplace = True)

    #Encode the variables via dummies
    user_data = pd.get_dummies(user_data, drop_first=True)
    #Interlay missing features based off train_df
    user_data = user_data.reindex(columns=train_df.columns, fill_value=False)

    #Transform the columns
    # For user data, scale numeric columns
    numeric_cols_val = user_data.select_dtypes(include=['float64', 'int64']).columns
    boolean_cols_train = user_data.select_dtypes(include=['bool']).columns

    scaled_numeric_val = transformer_scaler.transform(user_data[numeric_cols_val])

    # Create a DataFrame with the scaled numeric data for user data
    scaled_numeric_val_df = pd.DataFrame(scaled_numeric_val, columns=numeric_cols_val)

    # Combine the scaled numeric DataFrame with the original boolean DataFrame for user data
    user_data = pd.concat([scaled_numeric_val_df, user_data[boolean_cols_train].reset_index(drop=True)], axis=1)

    #Apply PCA transform
    user_data_pca_X = transformer_pca.transform(user_data)

    #Converts the pca_data into a dataframe, similar to what was passed for easy manipulation.
    columns = [f"Column {i+1}" for i in range(user_data_pca_X.shape[1])]
    user_data_pca_X = pd.DataFrame(user_data_pca_X, columns=columns)
    pca_data_df = pd.concat([time,user_data_pca_X,y],axis=1)
    return pca_data_df

In [19]:
# Load datasets
def load_dataset(path):
    df = pd.read_csv(path)
    if "time" in df.columns:
        df["time"] = pd.to_datetime(df["time"], utc = True)
    return df

def manipulate_datasets(energy_df, weather_df):
    #Energy Features
    energy_df["time"] = pd.to_datetime(energy_df["time"], utc = True)
    weather_df["time"] = pd.to_datetime(weather_df["time"], utc = True)
    energy_generation_columns = ['generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil gas', 'generation fossil hard coal',
       'generation fossil oil', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation nuclear',
       'generation other', 'generation other renewable', 'generation solar',
       'generation waste', 'generation wind onshore'
]
    energy_df["total_gen"] = energy_df[energy_generation_columns].sum(axis=1)

    #Weather data features creation
    weather_df["temp_diff"] = weather_df["temp_max"] - weather_df["temp_min"]
    weather_df.drop(columns=["temp_max","temp_min"],inplace=True)
    #Combine the dfs
    cities = weather_df["city_name"].unique()
    user_data = energy_df.copy()
    for city in cities:
        #indexes values for a certain city name and drops the column
        df_city = weather_df[weather_df["city_name"] == city]
        df_city = df_city.drop(columns="city_name")
        
        #Duplicate times still exist within the weather dataset with a unique city. To gain a unique time, a 
        numeric_cols = df_city.select_dtypes(include=['number']).columns
        
        #Assigns the mean of duplicate time value entries to a unique time value
        df_city = df_city.groupby('time')[numeric_cols].agg('mean').reset_index()
        
        #Rename columns to include the city name
        for col in numeric_cols:
            df_city = df_city.rename(columns={col: col + "_" + city})
        
        #Merge the dfs
        user_data = user_data.merge(df_city, on="time", how="inner")
    #Drop columns as no correlation exists as noted in data exploration
    try:
        user_data.drop(['snow_3h_ Barcelona', 'snow_3h_Seville'], axis=1, inplace=True)
    except:
        raise st.error(f"Ensure all five cities are represented within the dataset")
    #Fill nan values with mean
    user_data = user_data.fillna(user_data.mean())

    return user_data

def create_engineered_features(df, feature_engineered_data): 
    #Time Feats
    warnings.filterwarnings("ignore")
    def create_time_features(df, create_time_period_feature = False, step = 6):
        #Creates time related features of month, day, hour
        time_feat_df = pd.DataFrame()
        time_feat_df = df[["time","price actual"]]

        time_feat_df["month"] = time_feat_df['time'].dt.month_name()
        time_feat_df["day"] = time_feat_df['time'].dt.day_name()
        time_feat_df["hour"] = time_feat_df["time"].dt.hour.astype('category')
        #Creates stepped features based on hour
        #using pd.getdummies creates 23 additional features, whereas hour features could be captured via a period reducing dimensionality
        if create_time_period_feature:
            for i in range(step, 24, step):
                time_feat_df[f"is_btwn_{i}_and_{i+step}"] = time_feat_df["hour"].isin(range(i, i + step)).astype(bool)

            time_feat_df.drop(columns=["hour"],inplace=True)
        
        return time_feat_df
    time_best_feat = create_time_features(df, create_time_period_feature=True,step = 6).drop(columns="price actual")

#Lag and MA feats
# Create time features
    lag_ma_feats = create_time_features(df)
    lag_ma_feats["year"] = lag_ma_feats["time"].dt.year
    lag_ma_feats["week"] = lag_ma_feats["time"].dt.isocalendar().week
    lag_ma_feats.set_index("time", inplace=True)

    # Compute weekly average and merge with previous years for comparison
   
    weekly_avg_last_year = lag_ma_feats.groupby(["year", "week"])["price actual"].mean().reset_index()
    weekly_avg_last_year["last_year_weekly_mean_price"] = weekly_avg_last_year.groupby("week")["price actual"].shift(1)
    # Determine if a "last year exists within data, if it does not fill with mean from df_train"   
    if df.loc[df.index[-1], "time"] - timedelta(days=365) not in df["time"].values:
        weekly_avg_last_year["last_year_weekly_mean_price"].fillna(feature_engineered_data.groupby(feature_engineered_data["time"].dt.isocalendar().week)["last_year_weekly_mean_price"].transform("mean"), inplace=True)
    else:
        weekly_avg_last_year["last_year_weekly_mean_price"].fillna(weekly_avg_last_year.groupby("week")["last_year_weekly_mean_price"].transform("mean"), inplace=True)

    # Create lagged features
    for lag in [24, 24*7]: 
        lag_ma_feats[f"prev_{'day' if lag == 24 else 'week'}_price"] = lag_ma_feats["price actual"].shift(lag)
        lag_ma_feats[f"prev_{'day' if lag == 24 else 'week'}_price"].fillna(lag_ma_feats["prev_day_price"].mean(), inplace=True)

    # Create moving averages
    for period in [1, 3, 7, 14, 30, 90]:
        lag_ma_feats[f"{period}_day_ma"] = lag_ma_feats["price actual"].rolling(f"{period}D", min_periods=1).mean()

    lag_ma_feats = lag_ma_feats.reset_index().merge(weekly_avg_last_year.drop(columns="price actual"), on=["year", "week"], how="outer")
    lag_ma_feats.drop(columns=['month', 'day','hour','year', 'week'], inplace=True)
    lag_ma_feats.drop(columns="price actual",inplace=True)
    #Create a combined dataset
    merged_time_lag_ma_feat = pd.merge(time_best_feat,lag_ma_feats,on="time",how="inner")

    df = df.merge(merged_time_lag_ma_feat,on="time",how="inner")
    
    #Create quadratic trend features
    df.reset_index(inplace=True)
    df.rename(columns={"index":"t"},inplace=True)
    df["t^2"] = df["t"]**2
    # Create dummy variablse for non-numeric values while avoid co-linearity
    #df = pd.get_dummies(df,drop_first=True)
    return df

def apply_transformations(user_data, train_df):
    #Int transformers
    transformer_scaler = joblib.load(r"transformers\scaler.pkl")
    transformer_pca = joblib.load(r"transformers\pca.pkl")

    #Create variables for storing time, response variable, and predictors
    time = user_data["time"]
    y = user_data["price actual"]
    user_data.drop(columns = ["time","price actual"], inplace = True)

    #Encode the variables via dummies
    user_data = pd.get_dummies(user_data, drop_first=True)
    #Interlay missing features based off train_df
    user_data = user_data.reindex(columns=train_df.columns, fill_value=False)

    #Transform the columns
    # For user data, scale numeric columns
    numeric_cols_val = user_data.select_dtypes(include=['float64', 'int64']).columns
    boolean_cols_train = user_data.select_dtypes(include=['bool']).columns

    scaled_numeric_val = transformer_scaler.transform(user_data[numeric_cols_val])

    # Create a DataFrame with the scaled numeric data for user data
    scaled_numeric_val_df = pd.DataFrame(scaled_numeric_val, columns=numeric_cols_val)

    # Combine the scaled numeric DataFrame with the original boolean DataFrame for user data
    user_data = pd.concat([scaled_numeric_val_df, user_data[boolean_cols_train].reset_index(drop=True)], axis=1)

    #Apply PCA transform
    user_data_pca_X = transformer_pca.transform(user_data)

    #Converts the pca_data into a dataframe, similar to what was passed for easy manipulation.
    columns = [f"Column {i+1}" for i in range(user_data_pca_X.shape[1])]
    user_data_pca_X = pd.DataFrame(user_data_pca_X, columns=columns)
    pca_data_df = pd.concat([time,user_data_pca_X,y],axis=1)
    return pca_data_df

In [5]:
weather_df = pd.read_csv(r"engineered_data\weather_data_val_manipulated.csv")
energy_df = pd.read_csv(r"engineered_data\energy_data_val_manipulated.csv")
combined_data = load_dataset(r"engineered_data\transformed_data_first_rev.csv")
engineered_features_data = load_dataset(r"engineered_data\feature_engineered_data.csv")
pca_data = load_dataset("engineered_data\pca_ml_ready_data.csv")
ml_data_X_train = load_dataset(r"engineered_data\feature_engineered_data_X_train.csv")

In [10]:
deploy_df = manipulate_datasets(energy_df.copy(),weather_df.copy())

In [11]:
len(deploy_df)

7013

In [12]:
#Determine if manipulate_datasets outputs the same columns and features as combined_data
compare_columns_df(deploy_df,combined_data)

Missing columns from deploy_df compared to target_df: set()
Number of missing columns from deploy_df compared to target_df: 0
Missing columns from target_df compared to deploy_df: set()
Number of missing columns from target_df compared to deploy_df: 0
Number of columns in deploy_df: 75
Number of columns in target_df: 75
The number of columns in deploy_df is equal to that in target_df.


In [20]:
deploy_feature_engineered_df = create_engineered_features(deploy_df.copy(),engineered_features_data.copy())

In [21]:
compare_columns_df(deploy_feature_engineered_df.drop(columns=["price actual", "time"]),engineered_features_data.copy())

Missing columns from deploy_df compared to target_df: {'price actual', 'time'}
Number of missing columns from deploy_df compared to target_df: 2
Missing columns from target_df compared to deploy_df: set()
Number of missing columns from target_df compared to deploy_df: 0
Number of columns in deploy_df: 89
Number of columns in target_df: 91
The number of columns in deploy_df is not equal to that in target_df.


In [43]:
deploy_feature_engineered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7013 entries, 0 to 7012
Data columns (total 99 columns):
 #   Column                                       Non-Null Count  Dtype              
---  ------                                       --------------  -----              
 0   t                                            7013 non-null   int64              
 1   time                                         7013 non-null   datetime64[ns, UTC]
 2   generation biomass                           7013 non-null   float64            
 3   generation fossil brown coal/lignite         7013 non-null   float64            
 4   generation fossil gas                        7013 non-null   float64            
 5   generation fossil hard coal                  7013 non-null   float64            
 6   generation fossil oil                        7013 non-null   float64            
 7   generation hydro pumped storage consumption  7013 non-null   float64            
 8   generation hydro run-of-rive

In [38]:
for p, q in zip(deploy_feature_engineered_df.drop(columns=["time", "price actual"]).columns, ml_data_X_train.columns):
    if p != q:
        print(p)



is_btwn_6_and_12
is_btwn_12_and_18
is_btwn_18_and_24
prev_day_price
prev_week_price
1_day_ma
3_day_ma
7_day_ma
14_day_ma
30_day_ma
90_day_ma
last_year_weekly_mean_price
t^2
month_July
month_June
month_March
month_May
month_November
month_October
month_September
day_Monday
day_Saturday
day_Sunday
day_Thursday
day_Tuesday
day_Wednesday


In [22]:
deploy_feature_engineered_pca = apply_transformations(deploy_feature_engineered_df.copy(),ml_data_X_train)

In [23]:
compare_columns_df(deploy_feature_engineered_pca,pca_data)

Missing columns from deploy_df compared to target_df: set()
Number of missing columns from deploy_df compared to target_df: 0
Missing columns from target_df compared to deploy_df: set()
Number of missing columns from target_df compared to deploy_df: 0
Number of columns in deploy_df: 61
Number of columns in target_df: 61
The number of columns in deploy_df is equal to that in target_df.


In [49]:
deploy_feature_engineered_pca

,time,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7,Column 8,Column 9,...,Column 47,Column 48,Column 49,Column 50,Column 51,Column 52,Column 53,Column 54,Column 55,price actual
0,2018-03-14 18:00:00+00:00,-4.274857,0.825348,4.641214,8.804337,-1.959678,-1.889252,2.955439,2.872822,1.500136,...,-0.187850,0.897823,-1.595161,-0.201287,1.885004,0.050559,0.196497,-0.144339,-0.369273,50.95
1,2018-03-14 19:00:00+00:00,-2.960907,0.772773,3.585117,7.192523,-0.706520,-0.902016,2.667317,2.342305,-0.053881,...,0.289333,0.702527,-0.529769,0.212551,1.805751,-0.078712,-0.148836,-0.160198,-0.435772,59.73
2,2018-03-14 20:00:00+00:00,-3.443623,0.035503,3.852856,6.984090,-1.501447,-1.348218,2.493640,2.403798,0.083945,...,-0.622341,1.600989,-0.244156,0.725674,0.909233,0.553918,0.612848,0.016350,-0.384135,50.89
3,2018-03-14 21:00:00+00:00,-3.583661,-0.725679,3.110287,4.842457,-1.032246,-1.228726,1.865396,3.186572,0.361537,...,-0.723466,0.966384,-0.136178,0.504955,1.021985,0.804953,0.823498,0.601754,-0.136876,45.94
4,2018-03-14 22:00:00+00:00,-5.375297,-0.435624,4.353312,6.264673,-2.233537,-0.621059,2.596650,1.793152,0.959746,...,-0.726433,0.757391,-0.894388,1.348849,1.225874,0.341591,1.140301,0.913555,-0.036593,30.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7008,2018-12-31 18:00:00+00:00,2.226959,-2.483943,-2.149679,-0.989503,2.225141,0.871963,-0.183559,1.154734,0.281908,...,0.751888,-0.287421,-0.673239,0.727310,1.134760,-0.088459,0.544819,-0.263466,-0.525234,77.02
7009,2018-12-31 19:00:00+00:00,2.088942,-2.801025,-2.163501,-1.203789,2.344032,1.196169,0.104872,0.640753,0.170250,...,1.119913,0.507457,-0.231988,0.630862,1.455573,-0.532136,0.544907,-0.318163,0.143101,76.16
7010,2018-12-31 20:00:00+00:00,1.689580,-3.428798,-1.900630,-0.896439,1.872678,1.700217,-0.691526,-0.325952,0.136372,...,1.499733,0.230813,-0.214316,1.082689,1.104597,-0.548814,1.096338,-0.314873,-0.186263,74.30
7011,2018-12-31 21:00:00+00:00,1.275254,-3.734704,-1.947809,-1.336982,1.604230,1.981913,-0.032176,-0.825763,0.074386,...,1.411348,0.777585,-0.141087,1.059724,0.987899,-0.606894,1.063430,-0.333171,-0.241595,69.89


In [27]:
model = joblib.load(r"models\best_model_linear_reg.pkl")

In [28]:
y_pred = model.predict(deploy_feature_engineered_pca.drop(columns=["time", "price actual"]))
y_val = deploy_feature_engineered_pca["price actual"]

In [29]:
np.mean(y_pred)

67.6105065223949